In [1]:
"""
Скрипт для парсинга объявлений недвижимости с сайта cian.ru.

Инструкция по запуску:
1-2. Как включить
3. Повторяйте пункты 1-2 пока не получите сообщение 'Запуск полностью успешен'
Если запуск не получился 10ый раз, то продолжать запуск нет смысла.

Возможные причины: 
ваш IP заблокирован cian.ru (особенно часты перебои ночью),
добавлена новая защита от парсинга или изменена конфигурация страницы cian.ru.

Описание алгоритма.
Инфо об объявлениях содержится в 2 строках HTML кода:
Str_SoupInfo1str - урезанная информация об объявлениях,
Str_SoupInfo2str - полная информация об объявлениях.

Алгоритм:
1. Находим кусок кода с нужной строкой,
2. Внутри найденного куска кода выделяем кусок кода по информации о нужном объявлении
3. Выделяем из этого куска кода нужную информацию по ключевым словам

"""
print(' ')

In [2]:
#Необходимые бибилиотеки
from bs4 import BeautifulSoup #для загрузки страницы
from requests.exceptions import HTTPError #для идентификации прогрузки капчи
import cloudscraper #для загрузки страницы
import time #для остановок при запуске парсера
import re #для поиска HTML по тексту
import httpx #для поиска HTML по тексту

#Библиотеки для работы с данными
import pandas as pd #для выгрузки данных в .xlsx

In [3]:
""" Блок для выгрузки информации из 2ой сторки. 
Список функций: 
find_nth_meet_inside_str(string, substring, n), 
"""

def find_nth_meet_inside_str(string, substring, n):
    """ Для поиска номера символа n-ой встречи подстроки внутри строки"""
    if (n == 1):
        return string.find(substring)
    else:
        return string.find(substring, find_nth_meet_inside_str(string, substring, n - 1) + 1)

def f(key, data):
    """Из строки формата "VAR1": Value1, "VAR2": Value2 выдергивает 
    значения по запросу f(Var)"""
    pattern = r'"{}":(\w+),'.format(key)
    match = re.search(pattern, data)
    if match:
        return match.group(1)
    else:
        return None
    
def fstr(key, data):
    """Из строки формата "VAR1": Value1, "VAR2": Value2 выдергивает 
    значения по запросу f(Var)"""
    pattern = r'"{}":"([^"]+)"'.format(key)
    match = re.search(pattern, data)
    if match:
        return match.group(1)
    else:
        return None   

def fcoord(key, data):
    """Из строки формата "VAR1": Value1, "VAR2": Value2 выдергивает 
    значения по запросу f(Var)"""
    pattern = r'"{}":({[^}]+})'.format(key)
    match = re.search(pattern, data)
    if match:
        return match.group(1)
    else:
        return None      
   
def fcoord(key, data):
    """Из строки формата "VAR1": Value1, "VAR2": Value2 выдергивает 
    значения по запросу f(Var)"""
    pattern = r'"{}":(\S+)'.format(key)
    match = re.search(pattern, data)
    if match:
        return match.group(1)
    else:
        return None


In [35]:
#url ='https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=35000000&minarea=200&minfloor=1&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=2&office_type%5B2%5D=3&office_type%5B3%5D=5&office_type%5B4%5D=7&region=1&sort=price_object_order&p=2' 
#print(url)


html = cloudscraper.create_scraper().get('https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=35000000&minarea=200&minfloor=1&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=2&office_type%5B2%5D=3&office_type%5B3%5D=5&office_type%5B4%5D=7&region=1&sort=price_object_order&p=2').text
soup = BeautifulSoup(html)
str_soup = str(soup)

#Находим кусок кода с нужной информацией  
quote = "\'" #для одинарной кавычки в строке
WhereInfoStart = str_soup.find('window._cianConfig['+quote+'legacy-commercial-serp-frontend'+quote+']')



Info2str = str_soup[WhereInfoStart:]
print('Положение: '+str(WhereInfoStart))
        
#Нарезаем этот кусок кода по разделителю  WordDelimiter 
WordDelimiter = ',"offerType":'
#Выдернем данные от position:1 до position:2
#Найдем координаты всех position:
CurrentPosition = 1
WordDelimiterPositions = [1]
while CurrentPosition != -1:
    CurrentPosition = Info2str.find(WordDelimiter, CurrentPosition+len(WordDelimiter)) 
    WordDelimiterPositions.append(CurrentPosition)
#Заменим последний элемент на конец Info2str  
WordDelimiterPositions.remove(-1)
WordDelimiterPositions.append(len(Info2str)-1)
print('Кол-во объявлений: '+str(len(WordDelimiterPositions)-2))





Положение: 576195
Кол-во объявлений: 28


In [ ]:
Factors = ['№(Страница)_№(Объявления)', 'Id Продавца', 'Стоимость',  'Год постройки']
Factors.extend(['Площадь', 'Тип сделки', 'Этаж', 'Этажность', 'Высота потолка', 'Строка с координатами', 'Строка с районом'])
df3 = pd.DataFrame(columns=Factors)

#Нарежем str_soup по этим координатам   
for i in range(0, len(WordDelimiterPositions)-1):
    PositionData = Info2str[WordDelimiterPositions[i]:WordDelimiterPositions[i+1]]   

    tmp_for_print = []    
    tmp_for_print.append(str(1)+'_'+str(i))
    tmp_for_print.append(f('cianUserId', Info2str[WordDelimiterPositions[i]:WordDelimiterPositions[i+1]]))
    tmp_for_print.append(f('price', Info2str[WordDelimiterPositions[i]:WordDelimiterPositions[i+1]]))
    tmp_for_print.append(f('yearBuilt', Info2str[WordDelimiterPositions[i-1]:WordDelimiterPositions[i]]))
    tmp_for_print.append(fstr('totalArea', Info2str[WordDelimiterPositions[i]:WordDelimiterPositions[i+1]]))
    tmp_for_print.append(fstr('dealType', Info2str[WordDelimiterPositions[i]:WordDelimiterPositions[i+1]]))
    tmp_for_print.append(f('floor', PositionData))
    tmp_for_print.append(f('totalFloors', PositionData))
    tmp_for_print.append(f('ceilingHeight', PositionData))       
    if  fcoord('coordinates', PositionData)!= None:
        tmp_for_print.append(fcoord('coordinates', PositionData)[:40])
    else:
        tmp_for_print.append('None')

    tmp = PositionData[PositionData.find('"districts":[{"'):PositionData.find('"districts":[{"')+500]
    tmp_for_print.append(fstr('title', tmp))

    if str(f('cianUserId', PositionData)) != None:
        df3.loc[len(df3)] = tmp_for_print

display(df3)

In [37]:
Info2str

'window._cianConfig[\'legacy-commercial-serp-frontend\'] = (window._cianConfig[\'legacy-commercial-serp-frontend\'] || []).concat([{"key":"projectName","value":"legacy-commercial-serp-frontend","priority":1000,"filter":{}},{"key":"mcsRunEnv","value":"PROD","priority":1000,"filter":{}},{"key":"mcsRunEnvType","value":"prod","priority":1000,"filter":{}},{"key":"version","value":"[legacy-commercial-serp-frontend][CD-165980][3d262a1f]","priority":1000,"filter":{"request-id":"c9c521b0444b"}},{"key":"version.commit","value":"3d262a1f","priority":1000,"filter":{"request-id":"c9c521b0444b"}},{"key":"logger.browser.baseUrl","value":"https:\\u002F\\u002Fwww.cian.ru\\u002F","priority":52,"filter":{}},{"key":"logger.browser.pathApi","value":"\\u002Fajax\\u002Flog-error\\u002F","priority":6,"filter":{}},{"key":"logger.browser.errorFilter","value":["Failed get-infinite-search-result-desktop","Couldn\'t open socket connection, code: 1006","Недопустимый знак"],"priority":3,"filter":{}},{"key":"pathApi"

In [ ]:
soup.find('window._cianConfig['+quote+'legacy-commercial-serp-frontend'+quote+']')

In [31]:


html = cloudscraper.create_scraper().get('https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=35000000&minarea=200&minfloor=1&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=2&office_type%5B2%5D=3&office_type%5B3%5D=5&office_type%5B4%5D=7&region=1&sort=price_object_order&p=2' ).text

soup = BeautifulSoup(html)
str_soup = str(soup)
str_soup 

print(str_soup.find('window._cianConfig['+quote+'legacy-commercial-serp-frontend'+quote+']'))


print(str_soup.find('279494978'))
#Info2str

576284
7505


In [32]:
str_soup[576284:]

'window._cianConfig[\'legacy-commercial-serp-frontend\'] = (window._cianConfig[\'legacy-commercial-serp-frontend\'] || []).concat([{"key":"projectName","value":"legacy-commercial-serp-frontend","priority":1000,"filter":{}},{"key":"mcsRunEnv","value":"PROD","priority":1000,"filter":{}},{"key":"mcsRunEnvType","value":"prod","priority":1000,"filter":{}},{"key":"version","value":"[legacy-commercial-serp-frontend][CD-165980][3d262a1f]","priority":1000,"filter":{"request-id":"ac245cdd4436"}},{"key":"version.commit","value":"3d262a1f","priority":1000,"filter":{"request-id":"ac245cdd4436"}},{"key":"logger.browser.baseUrl","value":"https:\\u002F\\u002Fwww.cian.ru\\u002F","priority":52,"filter":{}},{"key":"logger.browser.pathApi","value":"\\u002Fajax\\u002Flog-error\\u002F","priority":6,"filter":{}},{"key":"logger.browser.errorFilter","value":["Failed get-infinite-search-result-desktop","Couldn\'t open socket connection, code: 1006","Недопустимый знак"],"priority":3,"filter":{}},{"key":"pathApi"

In [15]:
html = cloudscraper.create_scraper().get('https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=35000000&minarea=200&minfloor=1&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=2&office_type%5B2%5D=3&office_type%5B3%5D=5&office_type%5B4%5D=7&region=1&sort=price_object_order&p=2' ).text

soup = BeautifulSoup(html)
str_soup = str(soup)

#Находим кусок кода с нужной информацией  
quote = "\'" #для одинарной кавычки в строке
WhereInfoStart = str_soup.find('window._cianConfig['+quote+'legacy-commercial-serp-frontend'+quote+']')

print(WhereInfoStart)

Info2str = str_soup[WhereInfoStart:]

Info2str

576173


'window._cianConfig[\'legacy-commercial-serp-frontend\'] = (window._cianConfig[\'legacy-commercial-serp-frontend\'] || []).concat([{"key":"projectName","value":"legacy-commercial-serp-frontend","priority":1000,"filter":{}},{"key":"mcsRunEnv","value":"PROD","priority":1000,"filter":{}},{"key":"mcsRunEnvType","value":"prod","priority":1000,"filter":{}},{"key":"version","value":"[legacy-commercial-serp-frontend][CD-165980][3d262a1f]","priority":1000,"filter":{"request-id":"2ff4979f723"}},{"key":"version.commit","value":"3d262a1f","priority":1000,"filter":{"request-id":"2ff4979f723"}},{"key":"logger.browser.baseUrl","value":"https:\\u002F\\u002Fwww.cian.ru\\u002F","priority":52,"filter":{}},{"key":"logger.browser.pathApi","value":"\\u002Fajax\\u002Flog-error\\u002F","priority":6,"filter":{}},{"key":"logger.browser.errorFilter","value":["Failed get-infinite-search-result-desktop","Couldn\'t open socket connection, code: 1006","Недопустимый знак"],"priority":3,"filter":{}},{"key":"pathApi","

In [13]:
str_soup[WhereInfoStart:].find('frontend-mobile-website')

-1

In [9]:
WhereInfoStart 

-1

In [6]:

quote = "\'"
str_soup.find('window._cianConfig['+quote+'legacy-commercial-serp-frontend'+quote+']')

576246

In [ ]:
'window._cianConfig['+quote+'legacy-commercial-serp-frontend'+quote+']'

In [ ]:
quote = "\'"

In [ ]:
quote

In [ ]:
#Парсер основной

Factors = ['№(Страница)_№(Объявления)', 'Id Продавца', 'Стоимость',  'Год постройки']
Factors.extend(['Площадь', 'Тип сделки', 'Этаж', 'Этажность', 'Высота потолка', 'Строка с координатами', 'Строка с районом'])
#Factors.extend(['Продавец владелец?'])

df3 = pd.DataFrame(columns=Factors)
        
for j in range(1, 3):
    print('Загрузка страницы номер '+str(j))
    
    time.sleep(4) #спим 
    url = 'https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=35000000&minarea=200&minfloor=1&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=2&office_type%5B2%5D=3&office_type%5B3%5D=5&office_type%5B4%5D=7&region=1&sort=price_object_order&p=' 
 
    #Загрузка всего HTML кода страницы
    url = url+str(j)
    
    
    session = cloudscraper.create_scraper()
    for i in range(5):
        try:   
            print(url)
            res = session.get(url)           
            html = res.text
            soup = BeautifulSoup(html, 'lxml')
            #место выделения информации
            str_soup = str(soup)
            #============================
            #Парсим данные второй строки
            #============================
            
            
            #Находим кусок кода с нужной информацией    
            WhereInfoStart = find_nth_meet_inside_str(str_soup, 'window._cianConfig = window._cianConfig || {};', 3)
            WhereInfoFinish = find_nth_meet_inside_str(str_soup, 'function (w) {', 1)
            Info2str = str_soup[WhereInfoStart:WhereInfoFinish]

            #Проверим наличие 'frontend-mobile-website' в коде страницы
            if Info2str[1:300].find('frontend-mobile-website') !=-1:
                print('Успешная загрузка 2ой строки страницы')
            else:
                print('Ошибка при загрузке 2ой строки  страницы')


            #Нарезаем этот кусок кода по разделителю  WordDelimiter 
            WordDelimiter = ',"commercialInfo":'
            #Выдернем данные от position:1 до position:2
            #Найдем координаты всех position:
            CurrentPosition = 1
            WordDelimiterPositions = [1]
            while CurrentPosition != -1:
                CurrentPosition = Info2str.find(WordDelimiter, CurrentPosition+len(WordDelimiter)) 
                WordDelimiterPositions.append(CurrentPosition)
            #Заменим последний элемент на конец Info2str  
            WordDelimiterPositions.remove(-1)
            WordDelimiterPositions.append(len(Info2str)-1)
            #WhereWordDelimiterline2
            
            
            
            
            


            #Нарежем str_soup по этим координатам   
            for i in range(0, len(WordDelimiterPositions)-1):
                PositionData = Info2str[WordDelimiterPositions[i]:WordDelimiterPositions[i+1]]   

                tmp_for_print = []    
                tmp_for_print.append(str(1)+'_'+str(i))
                tmp_for_print.append(f('cianUserId', Info2str[WordDelimiterPositions[i]:WordDelimiterPositions[i+1]]))
                tmp_for_print.append(f('price', Info2str[WordDelimiterPositions[i]:WordDelimiterPositions[i+1]]))
                tmp_for_print.append(f('yearBuilt', Info2str[WordDelimiterPositions[i-1]:WordDelimiterPositions[i]]))
                tmp_for_print.append(fstr('totalArea', Info2str[WordDelimiterPositions[i]:WordDelimiterPositions[i+1]]))
                tmp_for_print.append(fstr('dealType', Info2str[WordDelimiterPositions[i]:WordDelimiterPositions[i+1]]))
                tmp_for_print.append(f('floor', PositionData))
                tmp_for_print.append(f('totalFloors', PositionData))
                tmp_for_print.append(f('ceilingHeight', PositionData))       
                if  fcoord('coordinates', PositionData)!= None:
                    tmp_for_print.append(fcoord('coordinates', PositionData)[:40])
                else:
                    tmp_for_print.append('None')

                tmp = PositionData[PositionData.find('"districts":[{"'):PositionData.find('"districts":[{"')+500]
                tmp_for_print.append(fstr('title', tmp))

                if str(f('cianUserId', PositionData)) != None and len(fstr('description', PositionData))>50:
                    df3.loc[len(df3)] = tmp_for_print

            
        
            break
        except requests.exceptions.HTTPError as e:
            print(f"Attempt {i+1} failed. {e}")
    else:
        print("Failed to retrieve page after 5 attempts.")


display(df3)

In [ ]:
#Парсер #2
url = 'https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=35000000&minarea=200&minfloor=1&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=2&office_type%5B2%5D=3&office_type%5B3%5D=5&office_type%5B4%5D=7&region=1&sort=price_object_order&p=' 

Factors = ['Страница_НомерНаСтранице', 'Ссылка на Объявление', 'Id Продавца']
Factors.extend(['Стоимость', 'Кол-во фото', 'Под снос?'])
Factors.extend(['Продавец владелец?'])

df3 = pd.DataFrame(columns=Factors)
#Из строки формата "VAR1": Value1, "VAR2": Value2 выдергивает значения по запросу f(Var)
def f(key, data):
    pattern = r'"{}":(\w+),'.format(key)
    match = re.search(pattern, data)
    if match:
        return match.group(1)
    else:
        return None

        
for j in range(2, 3):
    print('Загрузка страницы номер '+str(j))
    
    time.sleep(4) #спим 
 
    #Загрузка всего HTML кода страницы
    url = url+str(j)
    print(url)
    session = cloudscraper.create_scraper()
    for i in range(5):
        try:
            
            res = session.get(url)
            
            html = res.text
            #print(html)
            soup = BeautifulSoup(html, 'lxml')
            #место выделения информации
            str_soup = str(soup)
            #============================
            #Парсим данные второй строки
            #============================
            #Поиск инфо о первом объявлении второй строки
            FindWordDelimiterline2 = '"cianId":'
            FindWordFinishline2 = '/*#'
            Start_line2 = 1
            Finish_line2 = str_soup.find(FindWordFinishline2)
            #Сохраним строку
            line2 = str_soup[Start_line2:Finish_line2]

            #Выдернем данные от position:1 до position:2
            #Найдем координаты всех position:
            Current = Start_line2
            k = 0
            WhereWordDelimiterline2 = [str_soup.find(FindWordDelimiterline2, Current)]
            while Current != -1:
                Current = line2.find(FindWordDelimiterline2, Current+len(FindWordDelimiterline2)) 
                WhereWordDelimiterline2.append(Current)
                k = k + 1
                if k ==500:
                    break
            WhereWordDelimiterline2.remove(-1)
            WhereWordDelimiterline2.append(WhereWordDelimiterline2[-1]+700)


            #Нарежем str_soup по этим координатам   
            for i in range(1, len(WhereWordDelimiterline2)-1):
                start_tmp = WhereWordDelimiterline2[i]
                finish_tmp = WhereWordDelimiterline2[i+1]
                PositionData = str_soup[start_tmp:finish_tmp]    

                tmp_for_print = []
                tmp_for_print.append(str(j)+'_'+str(i))
                tmp_for_print.append('https://www.cian.ru/sale/commercial/'+str(f("id", PositionData)))
                tmp_for_print.append(f("ownerCianId", PositionData))
                tmp_for_print.append(f("price", PositionData))
                tmp_for_print.append(f("photosCount", PositionData))
                tmp_for_print.append(f("podSnos", PositionData))
                tmp_for_print.append(f("owner", PositionData))

                print(tmp_for_print)
                
                if str(f("id", PositionData)) != 'None':
                    df3.loc[len(df3)] = tmp_for_print

            
        
            break
        except requests.exceptions.HTTPError as e:
            print(f"Attempt {i+1} failed. {e}")
    else:
        print("Failed to retrieve page after 5 attempts.")


display(df3)

In [ ]:
#Для названия файла
current_time_str = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime(time.time()))
#Выгрузка в файл эксель
df3.to_excel("Df"+current_time_str+".xlsx")

In [ ]:
df3['Стоимость'] = df3['Стоимость'].astype(float)

df3withoutoutlier = df3[abs(df3['Стоимость']) < 2*df3['Стоимость'].mean()].copy()
import numpy as np
from scipy import stats
df3withoutoutlier['Стоимость'].hist(bins=20)

In [ ]:
abs(df3withoutoutlier['Стоимость']) < 2*df3withoutoutlier['Стоимость'].mean()

# Черновик ---------------------------------------------------------------
# Черновик ---------------------------------------------------------------
# Черновик ---------------------------------------------------------------

In [ ]:
#Cамый базовый парсер
for j in range(1, 2):
    print('Загрузка страницы номер '+str(j))
    
    time.sleep(4) #спим 
 
    #Загрузка всего HTML кода страницы
    url = 'https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=35000000&minarea=200&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=2&office_type%5B2%5D=3&office_type%5B3%5D=5&office_type%5B4%5D=7&region=1&p=' 
    url = url+str(j)
    print(url)
    session = cloudscraper.create_scraper()
    for i in range(5):
        try:
            
            res = session.get(url)
            
            html = res.text
            #print(html)
            soup = BeautifulSoup(html)
            offers = soup.select("div[data-name='CommercialOfferCard']")
                #Выгрузка ссылок по тегу из информации об объявлениях
            for ind, block in enumerate(offers):   
                common_data = dict()
                print(ind)
                print(block.select("[data-name='CommercialTitle']")[0].get('href'))
                print(block.get_text())
            
        
            break
        except requests.exceptions.HTTPError as e:
            print(f"Attempt {i+1} failed. {e}")
    else:
        print("Failed to retrieve page after 5 attempts.")


In [ ]:


str_soup = str(soup)
#============================
#Парсим данные второй строки
#============================
#Поиск инфо о первом объявлении второй строки
FindWordDelimiterline2 = '"cianId":'
FindWordFinishline2 = '/*#'
Start_line2 = 1
Finish_line2 = str_soup.find(FindWordFinishline2)
#Сохраним строку
line2 = str_soup[Start_line2:Finish_line2]

#Выдернем данные от position:1 до position:2
#Найдем координаты всех position:
Current = Start_line2
k = 0
WhereWordDelimiterline2 = [str_soup.find(FindWordDelimiterline2, Current)]
while Current != -1:
    Current = line2.find(FindWordDelimiterline2, Current+len(FindWordDelimiterline2)) 
    WhereWordDelimiterline2.append(Current)
    k = k + 1
    if k ==500:
        break
WhereWordDelimiterline2.remove(-1)
WhereWordDelimiterline2.append(WhereWordDelimiterline2[-1]+700)
        
    
#Нарежем str_soup по этим координатам   
for i in range(1, len(WhereWordDelimiterline2)-1):
    start_tmp = WhereWordDelimiterline2[i]
    finish_tmp = WhereWordDelimiterline2[i+1]
    PositionData = str_soup[start_tmp:finish_tmp]    
    
    tmp_for_print = []
    tmp_for_print.append(i)
    tmp_for_print.append(f("id", PositionData))
    tmp_for_print.append(f("ownerCianId", PositionData))
    tmp_for_print.append(f("price", PositionData))
    tmp_for_print.append(f("photosCount", PositionData))
    tmp_for_print.append(f("podSnos", PositionData))
    tmp_for_print.append(f("owner", PositionData))

    print(tmp_for_print)
    


In [ ]:
import re
import time
import requests
from bs4 import BeautifulSoup
import cloudscraper

def f(key, data):
    pattern = r'"{}":(\w+),'.format(key)
    match = re.search(pattern, data)
    if match:
        return match.group(1)
    else:
        return None

for j in range(1, 11):
    print('Page number '+str(j))
    time.sleep(4)

    url = 'https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=35000000&minarea=200&minfloor=1&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=2&office_type%5B2%5D=3&office_type%5B3%5D=5&office_type%5B4%5D=7&region=1&p=' 
    url = url+str(j)
    print(url)
    session = cloudscraper.create_scraper()
    for i in range(5):
        try:            
            res = session.get(url)        
            html = res.text         
            soup = BeautifulSoup(html, 'lxml')
            str_soup = str(soup)

            FindWordDelimiterline2 = '"cianId":'
            FindWordFinishline2 = '/*#'
            Start_line2 = 1
            Finish_line2 = str_soup.find(FindWordFinishline2)
            line2 = str_soup[Start_line2:Finish_line2]
            Current = Start_line2
            k = 0
            WhereWordDelimiterline2 = [str_soup.find(FindWordDelimiterline2, Current)]
            while Current != -1:
                Current = line2.find(FindWordDelimiterline2, Current+len(FindWordDelimiterline2)) 
                WhereWordDelimiterline2.append(Current)
                k = k + 1
                if k ==500:
                    break
            WhereWordDelimiterline2.remove(-1)
            WhereWordDelimiterline2.append(WhereWordDelimiterline2[-1]+700)

            for i in range(1, len(WhereWordDelimiterline2)-1):
                start_tmp = WhereWordDelimiterline2[i]
                finish_tmp = WhereWordDelimiterline2[i+1]
                PositionData = str_soup[start_tmp:finish_tmp]
                
                # Extract data by factors
                price = f("price", PositionData)
                num_rooms = f("roomsCount", PositionData)
                location = f("address", PositionData)
                area = f("area", PositionData)
                apt_type = f("objectSubtype", PositionData)
                floor_num = f("floorNumber", PositionData)
                num_floors = f("floorsCount", PositionData)
                description = f("description", PositionData)
                contact_name = f("owner", PositionData)
                contact_phone = f("phone", PositionData)
                contact_email = f("email", PositionData)
                date_posted = f("creationDate", PositionData)

                # Print data
                tmp_for_print = []
                tmp_for_print.append(str(j)+'_'+str(i))
                tmp_for_print.append('https://www.cian.ru/sale/commercial/'+str(f("id", PositionData)))
                tmp_for_print.append(price)
                tmp_for_print.append(num_rooms)
                tmp_for_print.append(location)
                tmp_for_print.append(area)
               


In [ ]:
response = requests.post(
    'https://api.cian.ru/commercial-search-offers/desktop/v1/offers/get-offers/',
    cookies=cookies,
    headers=headers,
    json=json_data,
)

response.text